# IBS of 3 bars - Strategy - Backtesting 

### Import Library

In [86]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
from backtesting._plotting import set_bokeh_output
set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [87]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [88]:
data = dataset.copy()

In [89]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [90]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2024-11-25 14:25:00,1298.0,1298.3,1296.7,1297.0,5524
2024-11-25 14:30:00,1297.1,1297.1,1297.1,1297.1,161
2024-11-25 14:45:00,1298.4,1298.4,1298.4,1298.4,5627


In [91]:
def set_condition_1(r):
    cond = False
    if r['Volume'] > 1.49 * r['vol_avg']:
        cond = True
    return cond

def set_condition_2(r):
    cond = ''
    if r['ibs'] > 0.7:
        cond = 'long'
    elif r['ibs'] < 0.3:
        cond = 'short'
    return cond

def set_condition_3(r):
    cond = ''
    if r['RSI'] < 40:
        cond = 'long'
    elif r['RSI'] > 60:
        cond  = 'short'
    return cond

def get_signal(r):
    signal = ''
    if r['condition_1'] == True:
        if r['condition_2'] == 'short' and r['condition_3'] == 'short':
            signal = 'short'
        elif r['condition_2'] == 'long' and r['condition_3'] == 'long':
            signal = 'long'
    return signal

In [92]:
def prepare_data(data):
    data['max_5'] = data['High'].rolling(5).max()
    data['min_5'] = data['Low'].rolling(5).min()
    data['ibs'] = data.apply(lambda x: (-1.0 if (x["High"] == x["Low"]) else (x["Close"] - x["Low"]) / (x["High"] - x["Low"])), axis=1)
    data['vol_avg'] = data['Volume'].rolling(20).mean()
    data["RSI"] = ta.rsi(data["Close"], length=14)
    data['condition_1'] = data.apply(lambda r: set_condition_1(r), axis=1)
    data['condition_2'] = data.apply(lambda r: set_condition_2(r), axis=1)
    data['condition_3'] = data.apply(lambda r: set_condition_3(r), axis=1)
    data['signal'] = data.apply(lambda r: get_signal(r), axis=1)
    return data

In [93]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [94]:
prepared_data

,Open,High,Low,Close,Volume,max_5,min_5,ibs,vol_avg,RSI,condition_1,condition_2,condition_3,signal
Date,,,,,,,,,,,,,,
2020-11-02 10:35:00,899.6,900.4,899.1,899.4,1106,901.5,898.8,0.230769,1810.20,48.103661,False,short,,
2020-11-02 10:40:00,899.2,899.4,898.8,899.4,1652,901.2,898.8,1.000000,1797.30,48.103661,False,long,,
2020-11-02 10:45:00,899.4,900.2,896.7,897.9,3197,900.4,896.7,0.342857,1873.65,39.673015,True,,long,
2020-11-02 10:50:00,897.8,898.9,896.8,898.3,2997,900.4,896.7,0.714286,1957.05,42.563838,True,long,,
2020-11-02 10:55:00,898.0,898.3,897.2,897.3,1668,900.4,896.7,0.090909,1954.35,37.700031,False,short,long,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-25 14:25:00,1298.0,1298.3,1296.7,1297.0,5524,1298.7,1295.9,0.187500,4150.55,48.873066,False,short,,
2024-11-25 14:30:00,1297.1,1297.1,1297.1,1297.1,161,1298.7,1295.9,-1.000000,3963.90,49.314332,False,short,,
2024-11-25 14:45:00,1298.4,1298.4,1298.4,1298.4,5627,1298.7,1296.2,-1.000000,4243.35,54.778494,False,short,,


In [95]:
prepared_data[prepared_data.signal != '']

,Open,High,Low,Close,Volume,max_5,min_5,ibs,vol_avg,RSI,condition_1,condition_2,condition_3,signal
Date,,,,,,,,,,,,,,
2020-11-02 11:00:00,897.3,897.4,895.4,897.0,3418,900.2,895.4,0.800000,1978.30,36.357765,True,long,long,long
2020-11-04 14:00:00,916.6,918.1,916.4,916.7,5942,918.1,912.4,0.176471,2442.80,67.446991,True,short,short,short
2020-11-06 10:55:00,908.2,909.6,908.0,909.5,1913,911.1,907.1,0.937500,1234.20,38.291924,True,long,long,long
2020-11-09 14:20:00,921.0,922.2,920.9,921.2,6445,922.2,916.3,0.230769,2190.30,66.567163,True,short,short,short
2020-11-09 14:45:00,923.0,923.0,923.0,923.0,5283,923.0,916.7,-1.000000,2492.30,72.536664,True,short,short,short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-15 11:20:00,1278.5,1280.3,1277.6,1279.7,6107,1283.3,1277.6,0.777778,4076.50,25.862475,True,long,long,long
2024-11-18 13:55:00,1272.2,1274.5,1271.6,1271.6,12142,1274.5,1266.3,0.000000,4508.10,61.672811,True,short,short,short
2024-11-18 14:25:00,1278.8,1278.8,1274.2,1275.0,10969,1280.3,1270.1,0.173913,6449.90,60.250140,True,short,short,short


In [96]:
class MainStrategy(Strategy):
    max_sl = 3.1
    trailing_sl = 4.5
    tp_step = 12
    def init(self):
        super().init()

    def next(self):
        super().next()
        _time = self.data.index
        current_time = _time[-1]
        if current_time.hour == 14 and current_time.minute >= 25:
            if self.position.is_long or self.position.is_short:
                self.position.close()
                return
                
        if current_time.hour == 14 and current_time.minute >= 30:
            return

        close_price = self.data.Close[-1]
        if self.position.is_long:
            max_5 = self.data.max_5[-1]
            if close_price < max_5 - self.trailing_sl:
                self.position.close()
        elif self.position.is_short:
            min_5 = self.data.min_5[-1]
            if close_price > min_5 + self.trailing_sl:
                self.position.close()

        if self.position:
            return  
        signal = self.data.signal[-1]
        if signal == 'long':
            buy_price = close_price
            sl = buy_price - self.max_sl
            tp = buy_price + self.tp_step
            self.buy(size=1, sl=sl, tp=tp)
        elif signal == 'short':
            sell_price = close_price
            sl = sell_price + self.max_sl
            tp = sell_price - self.tp_step
            self.sell(size=1, sl=sl, tp=tp)

In [97]:
bt = Backtest(prepared_data, MainStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()

In [98]:
stats

Start                     2020-11-02 10:35:00
End                       2024-11-26 09:05:00
Duration                   1484 days 22:30:00
Exposure Time [%]                    9.559406
Equity Final [$]                   9561.48561
Equity Peak [$]                   10005.33093
Return [%]                          -4.385144
Buy & Hold Return [%]               44.340672
Return (Ann.) [%]                   -1.107015
Volatility (Ann.) [%]                0.543148
Sharpe Ratio                        -2.038146
Sortino Ratio                       -2.849951
Calmar Ratio                        -0.246603
Max. Drawdown [%]                    -4.48906
Avg. Drawdown [%]                   -0.906211
Max. Drawdown Duration     1484 days 19:15:00
Avg. Drawdown Duration      296 days 23:34:00
# Trades                                  806
Win Rate [%]                        33.746898
Best Trade [%]                       1.241304
Worst Trade [%]                     -0.705464
Avg. Trade [%]                    

In [99]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,1,6,26,897.16907,898.0,0.83093,0.000926,2020-11-02 11:05:00,2020-11-02 14:10:00,None,0 days 03:05:00
1,-1,127,128,916.52496,917.1,-0.57504,-0.000627,2020-11-04 14:05:00,2020-11-04 14:10:00,None,0 days 00:05:00
2,1,209,213,909.97291,906.4,-3.57291,-0.003926,2020-11-06 11:00:00,2020-11-06 11:20:00,None,0 days 00:20:00
3,-1,284,285,920.72370,921.9,-1.17630,-0.001278,2020-11-09 14:25:00,2020-11-09 14:30:00,None,0 days 00:05:00
4,1,336,337,926.27780,921.0,-5.27780,-0.005698,2020-11-10 14:30:00,2020-11-10 14:45:00,None,0 days 00:15:00
...,...,...,...,...,...,...,...,...,...,...,...
801,1,51441,51457,1280.18394,1280.8,0.61606,0.000481,2024-11-15 11:25:00,2024-11-15 14:10:00,None,0 days 02:45:00
802,-1,51506,51507,1271.31849,1272.5,-1.18151,-0.000929,2024-11-18 14:00:00,2024-11-18 14:05:00,None,0 days 00:05:00
803,-1,51512,51513,1274.61750,1273.2,1.41750,0.001112,2024-11-18 14:30:00,2024-11-18 14:45:00,None,0 days 00:15:00
804,-1,51651,51652,1271.71837,1275.1,-3.38163,-0.002659,2024-11-21 13:20:00,2024-11-21 13:25:00,None,0 days 00:05:00
